In [132]:
import pandas as pd
# from ydata_profiling import ProfileReport

In [133]:
df = pd.read_csv('csv/uspevaemost.csv', encoding='cp1251')


In [134]:
# df[df['Студент'] == '335871344E556D4577794B7536697046726178657A513D3D'].sort_values(by=['Учебный год', 'Семестр'])

In [135]:
df = df[df['Статус'] == 'Отчисленный']
df = df[df['Квалификация'] == 'Бакалавр']
df = df[df['Форма обучения'] == 'Очная']

In [136]:
df['sem_start'] = df['Учебный год'].str[:4]
df['sem_end'] = df['Учебный год'].str[5:]

df_years = df.groupby(['Студент']).agg({'sem_start': 'min', 'sem_end': 'max'})

df_kol_let = (df_years['sem_end']).astype('int16') - (df_years['sem_start']).astype('int16')
print(df_kol_let.shape)
(df_kol_let <= 4).sum()

df_kol_let = df_kol_let[df_kol_let > 4]


# df_kol_let = pd.read_csv('TempData.csv')



(1536,)


In [137]:
df_kol_let.to_frame()

df_kol_let.to_csv('TempData.csv', index=True)
df_kol_let = pd.read_csv('TempData.csv')

df_kol_let.drop(df_kol_let.columns[1], axis=1, inplace=True)

df = df.merge(df_kol_let, indicator=True, how='outer').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge']).drop_duplicates()




In [138]:
df.to_csv('TempData1.csv', index=False)

df.drop_duplicates(inplace=True)

df = df[~(df['Оценка'] == 'неудовлетворительно') | 
        ~(df.duplicated(subset=df.columns.difference(['Оценка']))) ]

df['Оценка'] = df['Оценка'].map({
    'отлично': 5,
    'хорошо': 4,
    'удовлетворительно': 3,
    'зачтено': 2, 
    'незачет': 1 
})

In [139]:
df.drop('Статус', axis=1, inplace=True)
df.drop('Группа', axis=1, inplace=True)
df.drop('Специальность', axis=1, inplace=True)
df.drop('Квалификация', axis=1, inplace=True)

In [140]:
df_max = df.groupby(['Студент', 'Учебный год', 'Семестр', 'Дисциплина']).agg({'Оценка': 'max'})
df_min = df.groupby(['Студент', 'Учебный год',  'Семестр', 'Дисциплина']).agg({'Оценка': 'min'})

# df.sort_values(by=['Студент',"Учебный год", 'Семестр'], inplace=True)

In [141]:
df_max

Оценка
Студент                                          Учебный год Семестр Дисциплина                                                
2B41566C72437872494D356F7673772B4731724C4D413D3D 2015/2016   1       Введение в инженерную деятельность                       2
                                                                     Иностранный язык (английский)                            2
                                                                     Информатика 1.1                                          2
                                                                     История                                                  4
                                                                     Математика 1.1                                           3
...                                                                                                                         ...
7A7A38456B32345073425950566354506C2F4E3949773D3D 2015/2016   1       Математика 1.1                                           1
                                                                     Начертательная геометрия и инженерная графика 1.3        1
                                                                     Основы ядерных технологий                                2
                                                                     Химия 1.2                                                4
                                                                     Элективные дисциплины по физической культуре и ...       2

[78866 rows x 1 columns]

In [142]:
df_min = df_min.groupby(['Студент', 'Учебный год', "Семестр"], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_max = df_max.groupby(['Студент', 'Учебный год', "Семестр"], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)

In [159]:
df = df_max.merge(df_min, on=['Студент', 'Учебный год', 'Семестр']).rename(columns={'1_x': '1_max', '2_x': '2_max', '3_x': '3_max', '4_x': '4_max', '5_x': '5_max', 
                                                                                    '1_y': '1_min', '2_y': '2_min', '3_y': '3_min', '4_y': '4_min', '5_y': '5_min'})

df.to_csv('test1.csv', index=True)
df = pd.read_csv('test1.csv')
df

,Студент,Учебный год,Семестр,1_max,2_max,3_max,4_max,5_max,1_min,2_min,3_min,4_min,5_min
0,2B41566C72437872494D356F7673772B4731724C4D413D3D,2015/2016,1,0,4,2,2,0,0,4,2,2,0
1,2B41566C72437872494D356F7673772B4731724C4D413D3D,2015/2016,2,0,7,1,2,0,0,7,1,2,0
2,2B41566C72437872494D356F7673772B4731724C4D413D3D,2016/2017,3,0,5,2,2,0,0,5,2,2,0
3,2B41566C72437872494D356F7673772B4731724C4D413D3D,2016/2017,4,0,5,1,4,0,0,6,1,3,0
4,2B41566C72437872494D356F7673772B4731724C4D413D3D,2017/2018,5,0,5,1,3,0,0,5,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8955,7A74794B6C504F644431394D5A652F517757433847413D3D,2017/2018,5,0,5,4,0,0,0,5,4,0,0
8956,7A74794B6C504F644431394D5A652F517757433847413D3D,2017/2018,6,0,5,3,1,0,0,5,3,1,0
8957,7A74794B6C504F644431394D5A652F517757433847413D3D,2018/2019,7,0,5,2,2,0,0,5,4,0,0
8958,7A74794B6C504F644431394D5A652F517757433847413D3D,2018/2019,8,0,3,0,3,0,0,3,0,3,0


In [158]:
df['Идентификатор года'] = df.groupby(['Студент', 'Учебный год']).cumcount() + 1

# Удалите столбец 'Учебный год', если он больше не нужен
df = df.drop(columns=['Учебный год'])

# Выведите результат
df.to_csv('test.csv', index=True)

In [151]:

# Создайте уникальные числовые идентификаторы для каждого студента
# df['Идентификатор студента'] = df['Студент'].astype('category').cat.codes + 1

# Замените годы на числовые идентификаторы
df['Учебный год'] = df.groupby(['Студент'])['Студент'].transform(lambda x: range(1, len(x) + 1))

# Удалите временный столбец 'Идентификатор студента', если он больше не нужен
# df = df.drop(columns=['Идентификатор студента'])

# Выведите результат
df

,Студент,Учебный год,Семестр,1_max,2_max,3_max,4_max,5_max,1_min,2_min,3_min,4_min,5_min,Идентификатор студента
0,2B41566C72437872494D356F7673772B4731724C4D413D3D,1,1,0,4,2,2,0,0,4,2,2,0,1
1,2B41566C72437872494D356F7673772B4731724C4D413D3D,2,2,0,7,1,2,0,0,7,1,2,0,1
2,2B41566C72437872494D356F7673772B4731724C4D413D3D,3,3,0,5,2,2,0,0,5,2,2,0,1
3,2B41566C72437872494D356F7673772B4731724C4D413D3D,4,4,0,5,1,4,0,0,6,1,3,0,1
4,2B41566C72437872494D356F7673772B4731724C4D413D3D,5,5,0,5,1,3,0,0,5,2,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8955,7A74794B6C504F644431394D5A652F517757433847413D3D,5,5,0,5,4,0,0,0,5,4,0,0,1413
8956,7A74794B6C504F644431394D5A652F517757433847413D3D,6,6,0,5,3,1,0,0,5,3,1,0,1413
8957,7A74794B6C504F644431394D5A652F517757433847413D3D,7,7,0,5,2,2,0,0,5,4,0,0,1413
8958,7A74794B6C504F644431394D5A652F517757433847413D3D,8,8,0,3,0,3,0,0,3,0,3,0,1413


In [145]:
df.to_csv('test.csv', index=True)